In [1]:
import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_selector, ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge, Lasso, ElasticNet 
from sklearn.model_selection import train_test_split, cross_val_score,GridSearchCV, cross_val_predict
from sklearn.metrics import mean_squared_error

In [15]:
train = pd.read_csv(r"C:\Users\tuke-\Desktop\1_GSB544_Computing_and_Machine_Learning\Final\Classification\data\CAH-201803-train.csv")

train = train.dropna()

test = pd.read_csv(r"C:\Users\tuke-\Desktop\1_GSB544_Computing_and_Machine_Learning\Final\Classification\data\CAH-201803-test.csv")

test = test.dropna()

In [16]:
print(train)

     id_num      Q1  Q2 political_affiliation            Q4  \
0         1    Male  53           Independent       Liberal   
1         5  Female  66           Independent  Conservative   
2         7  Female  58              Democrat       Liberal   
3         8    Male  55           Independent      Moderate   
4         9    Male  64            Republican  Conservative   
..      ...     ...  ..                   ...           ...   
164     325    Male  21            Republican  Conservative   
165     328  Female  41            Republican       Liberal   
166     329    Male  60            Republican  Conservative   
167     332  Female  51            Republican  Conservative   
168     335    Male  67            Republican      Moderate   

                      Q5     Q6   Q7   Q8   Q9                      Q10  \
0         College degree  Black   No   No   No  Yes, somewhat religious   
1           Some college  White  Yes   No  Yes      Yes, very religious   
2         College 

Accuracy Scores for Each Fold: [0.61764706 0.5        0.61764706 0.58823529 0.63636364]
Mean Accuracy: 0.5919786096256685
